# Fundamentals

To get familiar with the different gate operations, we provide some example calculations for one Grover Iteration with 32 assignments, where number 6 is our solution.

In [58]:
import numpy as np

In [142]:
def hadamard():
    return 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

In [143]:
def x_gate():
    return np.array([[0, 1], [1, 0]])

In [144]:
def multi_controlled_x_gate(k, i): # k control qubits, i is index of target qubit starting at 0
    gate = np.identity(2**k)
    
    gate[2**k-1, 2**k-1] = 0
    gate[2**k-1, 2**k-1-2**(k-i)] = 1
    gate[2**k-1-2**(k-i), 2**k-1-2**(k-i)] = 0
    gate[2**k-1-2**(k-i), 2**k-1] = 1
    
    return gate

multi_controlled_x_gate(3, 3) 

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0.]])

In [145]:
def tensor(A, B):
    n1, m1 = np.shape(A)
    n2, m2 = np.shape(B)
    
    new_matrix = np.zeros((n1*n2, m1*m2))
    
    for i in range(n1):
        for j in range(m1):
             new_matrix[i*n2:(i+1)*n2, j*m2:(j+1)*m2] = A[i, j] * B
                
    return new_matrix

In [146]:
def probabilities(state_vector):
    return state_vector**2

## Calculating one Grover Iteration

Initialization

In [172]:
n = 5 # number of variables

In [173]:
solutions = np.array([[6]])

In [174]:
state_vector = np.zeros(2**(n)) # with res qubit
state_vector[0] = 1 # |000> state

State preparation

In [175]:
state_prep = tensor(tensor(tensor(tensor(hadamard(), hadamard()), hadamard()), hadamard()), hadamard())

In [176]:
state_vector = state_prep @ state_vector

In [177]:
probabilities(state_vector)

array([0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125])

Phase flip of the solution |01100> 

In [178]:
oracle = np.identity(2**n) 

In [179]:
oracle[solutions - 1, solutions - 1] *= -1

In [180]:
oracle

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [181]:
state_vector = oracle @ state_vector

In [182]:
state_vector

array([ 0.1767767,  0.1767767,  0.1767767,  0.1767767,  0.1767767,
       -0.1767767,  0.1767767,  0.1767767,  0.1767767,  0.1767767,
        0.1767767,  0.1767767,  0.1767767,  0.1767767,  0.1767767,
        0.1767767,  0.1767767,  0.1767767,  0.1767767,  0.1767767,
        0.1767767,  0.1767767,  0.1767767,  0.1767767,  0.1767767,
        0.1767767,  0.1767767,  0.1767767,  0.1767767,  0.1767767,
        0.1767767,  0.1767767])

In [183]:
probabilities(state_vector)

array([0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125])

Diffusion operator

In [184]:
diffuser = tensor(tensor(tensor(tensor(hadamard(), hadamard()), hadamard()), hadamard()), hadamard())

In [185]:
diffuser = tensor(tensor(tensor(tensor(x_gate(), x_gate()), x_gate()), x_gate()), x_gate()) @diffuser

In [186]:
diffuser = tensor(np.identity(2**(n-1)), hadamard()) @ diffuser

In [187]:
diffuser = multi_controlled_x_gate(5, 5) @ diffuser

In [188]:
diffuser = tensor(np.identity(2**(n-1)), hadamard()) @ diffuser

In [189]:
diffuser = tensor(tensor(tensor(tensor(x_gate(), x_gate()), x_gate()), x_gate()), x_gate()) @ diffuser

In [190]:
diffuser = tensor(tensor(tensor(tensor(hadamard(), hadamard()), hadamard()), hadamard()), hadamard()) @ diffuser

In [191]:
np.round(diffuser, 2)

array([[ 0.94, -0.06, -0.06, ..., -0.06, -0.06, -0.06],
       [-0.06,  0.94, -0.06, ..., -0.06, -0.06, -0.06],
       [-0.06, -0.06,  0.94, ..., -0.06, -0.06, -0.06],
       ...,
       [-0.06, -0.06, -0.06, ...,  0.94, -0.06, -0.06],
       [-0.06, -0.06, -0.06, ..., -0.06,  0.94, -0.06],
       [-0.06, -0.06, -0.06, ..., -0.06, -0.06,  0.94]])

In [192]:
state_vector = diffuser @ state_vector

In [195]:
state_vector

array([-0.15467961, -0.15467961, -0.15467961, -0.15467961, -0.15467961,
       -0.508233  , -0.15467961, -0.15467961, -0.15467961, -0.15467961,
       -0.15467961, -0.15467961, -0.15467961, -0.15467961, -0.15467961,
       -0.15467961, -0.15467961, -0.15467961, -0.15467961, -0.15467961,
       -0.15467961, -0.15467961, -0.15467961, -0.15467961, -0.15467961,
       -0.15467961, -0.15467961, -0.15467961, -0.15467961, -0.15467961,
       -0.15467961, -0.15467961])

In [193]:
probabilities(state_vector)

array([0.02392578, 0.02392578, 0.02392578, 0.02392578, 0.02392578,
       0.25830078, 0.02392578, 0.02392578, 0.02392578, 0.02392578,
       0.02392578, 0.02392578, 0.02392578, 0.02392578, 0.02392578,
       0.02392578, 0.02392578, 0.02392578, 0.02392578, 0.02392578,
       0.02392578, 0.02392578, 0.02392578, 0.02392578, 0.02392578,
       0.02392578, 0.02392578, 0.02392578, 0.02392578, 0.02392578,
       0.02392578, 0.02392578])

In [194]:
np.round(probabilities(state_vector), 2)

array([0.02, 0.02, 0.02, 0.02, 0.02, 0.26, 0.02, 0.02, 0.02, 0.02, 0.02,
       0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02,
       0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02])